# Wellness 심리 상담 데이터에 대한 KoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [5]:
!nvidia-smi

Active code page: 65001


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Colab 디렉토리 아래 dialogLM 경로 확인**




In [3]:
!ls drive/'My Drive'/'Colab Notebooks'/

 BERT_X		      'fastprogress example.ipynb'    NarrativeKoGPT2
 Data		       KorQuAD-beginner		      ReforBERT
 dialogLM	       korquad-finetuing.ipynb
 EnlipleBERTFintuing   korquad-finetuing-ver2.ipynb


**필요 패키지 설치**

In [1]:
# %pip install -r C:\sqlite\mysql\code\AI\FINAL_project\dialogLM\dialogLM\requirements.txt
!pip install kogpt2-transformers
!pip install kobert-transformers
!pip install torch

Active code page: 65001Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s2s-ft 0.0.1 requires transformers<=2.10.0, but you have transformers 3.0.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached kobert_transformers-0.4.1-py3-none-any.whl (12 kB)
  Using cached kogpt2_transformers-0.3.0-py3-none-any.whl (4.6 kB)
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB 640.0 kB/s eta 0:00:03
     ------------ --------------------------- 0.6/1.9 MB 5.9 MB/s eta 0:00:01
     ---------------------------------------  1.9/1.9 MB 13.1 MB/s eta 0:00:01
     ---------------------------------------- 1.9/1.9 MB 11.9 MB/s eta 0:00:00
  Using cached kss-4.5.4-py3-none-any.whl
  Using cached flask-3.0.0-py3-none-any.whl (99 kB)
  Using cached Flask_RESTful-0.3.10-py2.py3-none-any.whl (26 kB)
  Using cached pecab-1.0.8-py3-none-any.whl
  Using cached emoji-1.2.0-py3-none-any.whl (131 kB)
  Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
  Using cached werkzeug-3.0.1-py3-none-any.whl (226 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (1

In [ ]:
!pip install tokenizers
!pip install transformers

## KoGPT-2 Training

**Path 추가**

### 2.1 import package

In [1]:
import sys
sys.path.append("C:\\sqlite\\mysql\\code\\AI\\FINAL_project\\dialogLM")

In [2]:
import os
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import dataloader
from dialogLM.dataloader.wellness import WellnessAutoRegressiveDataset
from dialogLM.model.kogpt2 import DialogKoGPT2

c:\sqlite\mysql\code\mynewenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.is_available()

False

### KoGPT2 Training for Wellness dataset

In [7]:
root_path='C:\\sqlite\\mysql\\code\\AI\\FINAL_project\\dialogLM\\dialogLM\\dataloader'
data_path = f"{root_path}\\wellness_dialog_for_autoregressive_train.txt"
checkpoint_path =f"{root_path}\\checkpoint"
save_ckpt_path = f"{checkpoint_path}\\kogpt2-wellnesee-auto-regressive.pth"

n_epoch = 5         # Num of Epoch
batch_size = 2      # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 100 # 학습 저장 주기
learning_rate = 5e-5  # Learning Rate

dataset= WellnessAutoRegressiveDataset(data_path)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = DialogKoGPT2()
model.to(device)


loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses =[]
for epoch in range(n_epoch):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
            data = data.transpose(1, 0)
            data= data.to(ctx)

            outputs = model(data, labels=data)
            _, logits = outputs[:2]

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # if count % 10 == 0:
            #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
            if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss
                }, save_ckpt_path)
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

model.safetensors: 100%|██████████| 510M/510M [00:13<00:00, 37.1MB/s] 
c:\sqlite\mysql\code\mynewenv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--taeminlee--kogpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Train(0):   4%|▍         | 94/2360 [14:50<5:57:

KeyboardInterrupt: 